In [1]:
import pandas as pd
from pandas import TimeSeries
import numpy as np
import math
from bokeh.charts import TimeSeries, output_file, show
import statsmodels.tsa.stattools as stat

In [2]:
sp500=pd.read_csv('C:/mine/Course/9894/project/data/sp_500_daily_prices.csv',usecols=['date','COMNAM','PRC',\
                    'RET','RETX','vwretd','vwretx'],engine='c')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
sp_recent=sp500[sp500['date']>=20000103]

In [4]:
company_recent=list(sp_recent[sp_recent['date']==20000103]['COMNAM'])

In [5]:
sp_orig=sp_recent[sp_recent['COMNAM']==company_recent[-1]][['date','RET']]
sp_orig=pd.DataFrame(list(sp_orig['RET']),columns=[company_recent[-1]],index=list(sp_orig['date']))
sp_orig=sp_orig[~sp_orig.index.duplicated(keep='first')]

In [6]:
for company in company_recent:
    current_data=sp_recent[sp_recent['COMNAM']==company][['date','RET']]
    ret=pd.DataFrame(list(current_data['RET']),columns=[company],index=list(current_data['date']))
    ret =ret[~ret.index.duplicated(keep='first')]
    sp_orig=pd.concat([sp_orig,ret],join='outer',axis=1)

In [7]:
sp_new=sp_orig.loc[20000103:20100102,:]

In [8]:
sp_new=sp_new.dropna(axis=1)

In [9]:
sp_price=sp_recent[sp_recent['COMNAM']==company_recent[-1]][['date','PRC']]
sp_price=pd.DataFrame(list(sp_price['PRC']),columns=[company_recent[-1]],index=list(sp_price['date']))
sp_price=sp_price[~sp_price.index.duplicated(keep='first')]
for company in company_recent:
    current_data=sp_recent[sp_recent['COMNAM']==company][['date','PRC']]
    ret=pd.DataFrame(list(current_data['PRC']),columns=[company],index=list(current_data['date']))
    ret =ret[~ret.index.duplicated(keep='first')]
    sp_price=pd.concat([sp_price,ret],join='outer',axis=1)
sp_price=sp_price.loc[20000103:20100102,:]
#sp_price=sp_price.dropna(axis=1)

In [10]:
error_return=['E','D','C','B','A','']

In [11]:
sp_new=sp_new.applymap(lambda x:np.float64(0.0) if x in error_return else np.float64(x))

In [12]:
sp_pca=np.array(sp_new)

In [13]:
cov_company=np.cov(sp_pca)

In [14]:
cov_company[np.isnan(cov_company)] = 0.0

In [15]:
eigen_val,eigen_vector=np.linalg.eig(cov_company)

In [16]:
np.argmax(eigen_val)

0

In [17]:
eigen_vector_max=list(zip(eigen_vector[0],sp_new.columns))

In [18]:
eigen_vector_max.sort(key=lambda x:x[0],reverse=True)

In [19]:
eigen_vector_max=list(zip(*eigen_vector_max))

In [26]:
def find_cointegrated_pairs(data):
    n = len(data.columns)
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = list(data.columns)
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = np.array(data[keys[i]])
            S2 = np.array(data[keys[j]])
            result = stat.coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.01:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs

In [27]:
data=sp_price[list(eigen_vector_max[1][:20])]

In [28]:
score_matrix, pvalue_matrix, pairs=find_cointegrated_pairs(data)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:1546: RuntimeWarning: invalid value encountered in greater
  return sum(S > tol)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1731: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


In [29]:
len(pairs)

23

In [310]:
eigen_vector_max[1]

('K L A TENCOR CORP',
 'BANK OF AMERICA CORP',
 'ECOLAB INC',
 'X L CAPITAL LTD',
 'FEDERAL HOME LOAN MORTGAGE CORP',
 'PEPSI BOTTLING GROUP INC',
 'MOLEX INC',
 'AMERICAN EXPRESS CO',
 'FASTENAL COMPANY',
 'T J X COMPANIES INC NEW',
 'MAXIM INTEGRATED PRODUCTS INC',
 'EOG RESOURCES INC',
 'ALEXION PHARMACEUTICALS INC',
 'APPLIED MICRO CIRCUITS CORP',
 'CERNER CORP',
 'HUDSON CITY BANCORP INC',
 'EMERSON ELECTRIC CO',
 'WASHINGTON POST CO',
 'QUALCOMM INC',
 'CONEXANT SYSTEMS INC',
 'BRISTOL MYERS SQUIBB CO',
 'MCCORMICK & CO INC',
 'ROPER INDUSTRIES INC NEW',
 'MOTOROLA INC',
 'ABBOTT LABORATORIES',
 'FRANKLIN RESOURCES INC',
 'SOUTHWESTERN ENERGY CO',
 'ALCOA INC',
 'MCGRAW HILL COS INC',
 'PHILLIPS VAN HEUSEN CORP',
 'RAYTHEON CO',
 'EQUIFAX INC',
 'SUN MICROSYSTEMS INC',
 'OMNICOM GROUP INC',
 'COCA COLA CO',
 'OFFICE DEPOT INC',
 'CORNING INC',
 'BROWN SHOE CO INC NEW',
 'JONES APPAREL GROUP INC',
 'ROBERT HALF INTERNATIONAL INC',
 'WATERS CORP',
 'CARDINAL HEALTH INC',
 'QUESTAR 

In [98]:
return_company=[]
used_company=[]
for company in company_name:
    ret=list(map(lambda x:np.float64(0.0) if x in error_return else np.float64(x),list(sp_by_company.get_group(company)['RET'])))
    if len(ret)>1000:
        return_company+=[ret[:1000]]
        used_company+=[company]
    #print(company)

In [99]:
return_company=np.array(return_company)